In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests
import nltk

In [2]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [3]:
response.text[:1500]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with World Library, Inc., from their\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\noften releases Etexts that are NOT placed in the Public Domain!!\n\nShakespeare\n\n*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your

In [4]:
data = response.text.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [5]:
data = data[253:]
data[0]

'  From fairest creatures we desire increase,'

In [6]:
len(data)

124204

In [7]:
data = " ".join(data)
data[:1000]

"  From fairest creatures we desire increase,   That thereby beauty's rose might never die,   But as the riper should by time decease,   His tender heir might bear his memory:   But thou contracted to thine own bright eyes,   Feed'st thy light's flame with self-substantial fuel,   Making a famine where abundance lies,   Thy self thy foe, to thy sweet self too cruel:   Thou that art now the world's fresh ornament,   And only herald to the gaudy spring,   Within thine own bud buriest thy content,   And tender churl mak'st waste in niggarding:     Pity the world, or else this glutton be,     To eat the world's due, by the grave and thee.                        2   When forty winters shall besiege thy brow,   And dig deep trenches in thy beauty's field,   Thy youth's proud livery so gazed on now,   Will be a tattered weed of small worth held:     Then being asked, where all thy beauty lies,   Where all the treasure of thy lusty days;   To say within thine own deep sunken eyes,   Were an al

In [8]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [9]:
len(tokens)

898199

In [10]:
length = 30 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199971


In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [13]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]

array([  47, 1408, 1264,   37,  451, 1406,    9, 2766, 1158, 1213,  171,
        132,  269,   20,   24,    1, 4782,   87,   30,   98, 4781,   18,
        715, 1263,  171,  211,   18,  829,   20,   27])

In [14]:
vocab_size = len(tokenizer.word_index) + 1


In [15]:
y = to_categorical(y, num_classes=vocab_size)

In [16]:
vocab_size

13009

In [17]:
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
seq_length = X.shape[1]
seq_length

30

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 30, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 30)            390270    
_________________________________________________________________
lstm (LSTM)                  (None, 30, 100)           52400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 1,847,079
Trainable params: 1,847,079
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [22]:
model.fit(X, y, batch_size = 256, epochs = 50)

Epoch 1/50
782/782 [==============================] - 229s 293ms/step - loss: 6.8859 - accuracy: 0.0301
Epoch 2/50
782/782 [==============================] - 232s 296ms/step - loss: 6.5364 - accuracy: 0.0417
Epoch 3/50
782/782 [==============================] - 237s 303ms/step - loss: 6.3472 - accuracy: 0.0547
Epoch 4/50
782/782 [==============================] - 227s 290ms/step - loss: 6.1761 - accuracy: 0.0696
Epoch 5/50
782/782 [==============================] - 230s 294ms/step - loss: 6.0446 - accuracy: 0.0786
Epoch 6/50
782/782 [==============================] - 227s 290ms/step - loss: 5.9391 - accuracy: 0.0846
Epoch 7/50
782/782 [==============================] - 226s 290ms/step - loss: 5.8438 - accuracy: 0.0896
Epoch 8/50
782/782 [==============================] - 227s 291ms/step - loss: 5.7552 - accuracy: 0.0951
Epoch 9/50
782/782 [==============================] - 231s 295ms/step - loss: 5.6691 - accuracy: 0.0995
Epoch 10/50
782/782 [==============================] - 245s 313m

In [33]:
seed_text=lines[1]
seed_text

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to'

In [34]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [35]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

'wilful barks to qualify in ranks and wrinkles of the world and giddy censure and mischief thereto view and with a choir of weakness to th smothering of the castle enter king and ladies sovereign of the fusty is the remainder of the war and in the capitol of the castle enter marcius with drum and colours with artless absence and heat toward the pangs of twins appeard all the king and to the capitol exceed the antiates to be requires out of the world and to the sessions of the groundlings and seconded and we discovered unhand me world'